In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import pathlib

In [4]:
import xarray as xr
import dask
from dask.diagnostics import ProgressBar

### parameters for papermill 

In [5]:
dict_rename = {}
dict_rename['lon'] = 'longitudes'
dict_rename['lat'] = 'latitudes'

In [6]:
varname = 'PET'
first_year = 1972
last_year = 2021

In [7]:
dpath_vcsn = f"/media/nicolasf/END19101/data/VCSN/daily/{varname}"

In [8]:
dpath_vcsn = pathlib.Path(dpath_vcsn)

In [9]:
lfiles = list(dpath_vcsn.glob("*.nc"))

In [10]:
lfiles.sort()

In [11]:
lfiles

[]

In [12]:
vcsn_dset = xr.open_mfdataset(lfiles, concat_dim='time', combine='nested')

In [13]:
vcsn_dset

<xarray.Dataset>
Dimensions:  (time: 18761, lat: 257, lon: 241)
Coordinates:
  * time     (time) datetime64[ns] 1972-01-01 1972-01-02 ... 2023-05-13
  * lat      (lat) float64 -47.27 -47.23 -47.17 -47.12 ... -34.52 -34.48 -34.42
  * lon      (lon) float64 166.5 166.5 166.6 166.6 ... 178.3 178.4 178.4 178.5
Data variables:
    Tmin_n   (time, lat, lon) float64 dask.array<chunksize=(31, 257, 241), meta=np.ndarray>
    agent    (time, lat, lon) float32 dask.array<chunksize=(31, 257, 241), meta=np.ndarray>
    mask     (time, lat, lon) float64 dask.array<chunksize=(31, 257, 241), meta=np.ndarray>
Attributes:
    pyproj_srs:  +units=m +init=epsg:4272

In [14]:
vcsn_dset = vcsn_dset.chunk({'time':-1, 'lat':10, 'lon':10})

In [15]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    vcsn_dset = vcsn_dset.resample({'time':'1M'}).mean('time')

In [16]:
list(map(str, [first_year, last_year]))

['1972', '2021']

In [17]:
vcsn_dset = vcsn_dset.sel(time=slice(*list(map(str, [first_year, last_year]))))

In [18]:
vcsn_dset = vcsn_dset.rename(dict_rename)

In [19]:
with ProgressBar(): 
    vcsn_dset.to_netcdf(f'/media/nicolasf/END19101/data/PICT/datasets/VCSN/VCSN_monthly_{varname}_{first_year}_{last_year}_grid.nc')

[########################################] | 100% Completed | 21m 8ss


In [20]:
varname

'Tmin_n'